In [160]:
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import socket

In [161]:
my_url = 'https://finance.yahoo.com/topic/stock-market-news/'
#my_url = 'https://finance.yahoo.com/quote/XOM/news?p=XOM'

headers = {"User-Agent": "Mozilla/5.0"}
cookies = {"CONSENT": "YES+cb.20210720-07-p0.en+FX+410"}

response = Request(my_url, headers=headers)

In [162]:
def get_page(url):
    """Download a webpage and return a beautiful soup doc"""
    response = Request(url, headers=headers)
    page_content = urlopen(response).read() 
    doc = BeautifulSoup(page_content)
    return doc

In [163]:
doc = get_page(my_url)
doc.find('title')

<title>Latest Stock Market News</title>

In [164]:
div_tags = doc.find_all('div', {'class': "Ov(h) Pend(44px) Pstart(25px)"})

In [165]:
len(div_tags)

2

In [166]:
#print(div_tags[1])

In [167]:
#print("Source: ", div_tags[1].find('div').text)
#print("Head Line : {}".format(div_tags[1].find('a').text))

In [168]:
def get_news_tags(doc):
    """Get the list of tags containing news information"""
    news_class = "Ov(h) Pend(44px) Pstart(25px)" ## class name of div tag 
    news_list  = doc.find_all('div', {'class': news_class})
    return news_list

In [169]:
BASE_URL = 'https://finance.yahoo.com' #Global Variable 

def parse_news(news_tag):
    """Get the news data point and return dictionary"""
    news_source = news_tag.find('div').text #source
    news_headline = news_tag.find('a').text #heading
    news_url = news_tag.find('a')['href'] #link
    news_content = news_tag.find('p').text #content
    news_image = news_tag.findParent().find('img')['src'] #thumb image
    return { 'source' : news_source,
            'headline' : news_headline,
            'url' : BASE_URL + news_url,
            'content' : news_content,
            'image' : news_image
           }

In [170]:
import pandas as pd

In [171]:
def scrape_yahoo_news(url, path=None):
    """Get the yahoo finance market news and write them to CSV file """
    if path is None:
        path = 'stock-market-news.csv'
        
    print('Requesting html page')
    doc = get_page(url)

    print('Extracting news tags')
    news_list = get_news_tags(doc)

    print('Parsing news tags')
    news_data = [parse_news(news_tag) for news_tag in news_list]

    print('Save the data to a CSV')
    news_df = pd.DataFrame(news_data)
    news_df.to_csv(path, index=None)
    
    #This return statement is optional, we are doing this just analyze the final output 
    return news_df 

In [172]:
YAHOO_NEWS_URL = BASE_URL+'/topic/stock-market-news/'
news_df = scrape_yahoo_news(YAHOO_NEWS_URL)

Requesting html page
Extracting news tags
Parsing news tags
Save the data to a CSV


In [173]:
news_df.head()

,source,headline,url,content,image
0,Business,Stock Market Holidays 2022: Is Wall Street Clo...,https://finance.yahoo.com/m/ffc04835-c206-33ff...,Take a look at this list of stock market holid...,https://s.yimg.com/uu/api/res/1.2/ruyhBIrFwOrK...
1,Business,These Are The Top 5 China Stocks To Buy And Wa...,https://finance.yahoo.com/m/10c9e207-b808-3447...,Chinese stocks are rebounding with Covid curbs...,https://s.yimg.com/uu/api/res/1.2/x4Xx7YiE3YiX...
